In [1]:
def generator(n):
    return e^(-2*i*pi / (3*n));

In [2]:
class FFT:
    def generator(self,n):
        return e^(-2*i*pi / (3*n));
    
    def __init__(self,n):
        self.n = n;

        
        fac = factor(n);
    
        if len(fac) == 2:
            Radix2 = fac[0][1];
            Radix3 = fac[1][1];

        if len(fac) == 1:
            if fac[0][0] == 2:
                Radix2 = fac[0][1];
                Radix3 = 0;

            if fac[0][0] == 3:
                Radix2 = 0;
                Radix3 = fac[0][1];

        level = Radix2+Radix3;

        self.w = self.generator(n);
        self.omega = self.w^n;
    
        tree = zero_matrix(ZZ,level+1,n);
        tree[0,0] = 3*n;

        self.zetas = [self.w^0];
                
        #Radix-2 NTT with Cyclotomic Polynomial
        tree[1,0] = tree[0,0] / 6;
        tree[1,1] = 5*tree[0,0] / 6;

        x = self.w^(tree[1,0]);
        self.zetas.append(x);    
    
        #Radix-3 NTT
        for ll in range(1,Radix3+1):
            for ii in range(2*3^(ll-1)):
                tree[ll+1,3*ii  ] = tree[ll  ,  ii] / 3;
                tree[ll+1,3*ii+1] = tree[ll  ,  ii] / 3 + tree[0,0]/3;
                tree[ll+1,3*ii+2] = tree[ll  ,  ii] / 3 + 2*tree[0,0]/3;

                x = self.w^(tree[ll+1,3*ii]);
                self.zetas.append(x);
                x = self.w^(tree[ll+1,3*ii]*2);
                self.zetas.append(x);

        #Radix-2 NTT
        for ll in range(Radix3+1,level):
            for ii in range(2*3^(Radix3)*2^(ll-(Radix3+1))):
                tree[ll+1,2*ii  ] = tree[ll  ,  ii] / 2;
                tree[ll+1,2*ii+1] = tree[ll  ,  ii] / 2 + tree[0,0]/2;

                x = self.w^(tree[ll+1,2*ii]);
                self.zetas.append(x);

        print(tree)
        
    def fft(self, a):
        k = 1;

        b = a.copy();

        step = self.n/2;
        
        zeta = self.zetas[k];
        k = k+1;

        for ii in range(self.n/2):
            
            t = b[ii + step]*zeta;
            
            b[ii + step] = b[ii] - t + b[ii+step];
            b[ii       ] = b[ii] + t;
        
        for start in range(0,self.n, 384):
            zeta1 = self.zetas[k];
            k = k + 1;
            zeta2 = self.zetas[k];
            k = k + 1;

            for ii in range(start,start+128):
                t1 = zeta1 * b[ii+128];
                t2 = zeta2 * b[ii+256];
                t3 = self.omega * (t1-t2);
                
                b[ii+256] = b[ii] - t1 - t3;
                b[ii+128] = b[ii] - t2 + t3;
                b[ii    ] = b[ii] + t1 + t2;
                
        step = 64;

        while step >= 1:
            for start in range(0,self.n,(step << 1)):
                zeta = self.zetas[k];
                k += 1;

                for ii in range(start,start+step):
                    t = zeta * b[ii+step];

                    b[ii+step] = b[ii]-t;
                    b[ii     ] = b[ii]+t;
            
            step >>= 1;
            
            
        return b;

In [3]:
g = [-1, 0, -1, 0, 0, 0, -1, 0, 0, 1, -1, -1, 0, 1, -1, 0, -1, 1, 1, 1, -1, 1, 1, -1, -1, 0, -1, 0, 0, 0, 1, -1, -1, 1, 0, -1, 0, 0, 0, -1, -1, 0, 1, -1, 0, 1, -1, 1, 0, -1, -1, 0, 0, 0, -1, 1, 0, -1, 1, -1, 0, -1, 0, 1, -1, -1, 0, 1, 0, -1, -1, -1, 0, 0, -1, 0, 0, -1, -1, -1, 1, 0, -1, 0, 0, 1, 0, -1, 1, 0, -1, 0, -1, 0, 1, -1, 1, 0, -1, 0, -1, 0, 1, -1, -1, -1, 1, -1, 0, -1, -1, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 0, -1, -1, 1, 0, 1, 0, 0, -1, 0, 0, -1, 1, 1, 1, 0, 1, -1, 1, 0, 0, 1, 0, -1, 0, -1, -1, -1, -1, 0, -1, 1, 0, -1, 1, 1, 0, -1, -1, 0, 0, 1, 0, -1, 0, 1, -1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 1, 0, 1, -1, 0, -1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, -1, 1, -1, 1, 1, 0, 1, -1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 1, -1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, -1, -1, -1, -1, -1, 0, 0, -1, 0, 0, 0, -1, 1, -1, 0, 0, 0, 0, -1, 1, 1, 0, -1, 1, 1, 0, 1, -1, 1, 0, 0, 1, 0, 0, 0, -1, 0, -1, 0, -1, 0, 0, 1, 0, -1, 0, 0, 1, -1, 0, 0, -1, 0, -1, 0, 1, 1, 0, 0, 0, -1, 0, 1, 0, -1, 0, -1, 0, 0, -1, 1, -1, -1, 1, 1, 0, -1, 0, 1, -1, 0, 1, 1, 0, 0, -1, -1, 0, 0, 1, -1, 0, -1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, -1, -1, 1, 1, 1, 1, 0, 0, 0, 0, -1, 1, 0, 1, 1, -1, 0, 1, -1, 0, -1, -1, -1, -1, 0, 0, 1, 1, 1, 0, 0, 0, -1, 1, 1, 0, 1, 1, -1, 0, 0, -1, 0, 1, 0, -1, 0, -1, -1, -1, 0, 0, -1, 0, 0, 1, 0, 0, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, 0, 1, -1, 1, 0, 0, 0, 0, 0, -1, -1, 1, 1, 0, 0, 0, -1, 0, 1, 0, 1, -1, -1, -1, 1, 0, 0, 1, 0, -1, 0, 0, 0, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 0, 0, -1, 1, 0, 0, 1, 0, 0, -1, 0, 0, 0, -1, 1, -1, -1, -1, 0, 0, 0, -1, 0, -1, 0, 0, -1, -1, -1, 0, 1, 1, 0, 0, 1, 0, 0, -1, 1, 1, 0, 1, 1, 1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, -1, 0, 0, 0, 0, -1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, -1, 0, 0, 0, -1, 1, 1, 1, -1, 1, 0, 0, 0, 0, -1, -1, -1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 1, 1, -1, 0, 0, 1, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, -1, 0, 0, 0, -1, -1, 0, 0, 1, 0, -1, 0, 0, -1, 1, 0, 0, 0, 0, -1, 0, 0, -1, -1, 0, 0, 0, 0, -1, 0, 0, 1, 0, 1, -1, -1, 0, 0, 1, 0, 0, 1, 0, 1, -1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, -1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0]

In [4]:
fft = FFT(768)

[2304    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0

In [5]:
ghat = fft.fft(g)

In [12]:
ghat[384].imag().n(100)

8.6258408273835055212462676186

In [ ]:
(ghat[0].real()*65536).n(100)

In [59]:
n = 768

In [ ]:
w = generator(n);

In [ ]:
zeta = w^1153

In [ ]:
sss = 0;
for k in range(n):
    sss += g[i] * zeta^k;

In [21]:
sss.real().n(100)

0.00000000000000000000000000000

In [22]:
sss.imag().n(100)

0.00000000000000000000000000000

In [23]:
len(fft.zetas)

768

In [27]:
(fft.zetas[1].real()*65536).n(100)

32768.000000000000000000000000

In [28]:
(fft.zetas[2].imag()*65536).n(100)

-22414.632112991026088778112348

In [29]:
(fft.zetas[2]*65536).n(1000)

61583.6955958252918573701055987536016137393362871559541942050267315713612838266115817305466364197645491844962580292322313483516553305326925223219117171676668231614810237964429691157326096747931075304957891755754650167031047128535217695890426081259728841486932219297743449557542192136687213754758499541 - 22414.6321129910260887781123478165638848894315734080309470854183442654667539793575650387677897376102836124734934992699722063936326174286948430941616138601857010491981049899302057124706489132755726932083599361939389936712868415182357831362156572701763242713449957021562621101839254034355224982227203601*I

In [27]:
round(e^(-767/3*pi)*2^16)

0

In [28]:
i

I

In [29]:
def generator(n):
    return e^(-2*i*pi / (3*n));

In [30]:
z = generator(768)

In [31]:
(z^384 * 65536).real().n(100)

32768.000000000000000000000000

In [51]:
round(fft.zetas[767].real()*65536)

-65536

In [52]:
round(fft.zetas[767].imag()*65536)

-179